In [1]:
#This is for map transaction hover india state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/transaction/hover/country/india/state"
Map_trans_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'District_name':[], 'Transaction_count':[], 'Transaction_amount':[]}

for i in Map_trans_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['hoverDataList']:
              Name=z['name']
              count=z['metric'][0]['count']
              amount=z['metric'][0]['amount']
              clm['District_name'].append(Name)
              clm['Transaction_count'].append(count)
              clm['Transaction_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Map_trans_state=pd.DataFrame(clm)
print(Map_trans_state)

                           State  Year  Quater  \
0      andaman-&-nicobar-islands  2018       1   
1      andaman-&-nicobar-islands  2018       1   
2      andaman-&-nicobar-islands  2018       1   
3      andaman-&-nicobar-islands  2018       2   
4      andaman-&-nicobar-islands  2018       2   
...                          ...   ...     ...   
20599                west-bengal  2024       4   
20600                west-bengal  2024       4   
20601                west-bengal  2024       4   
20602                west-bengal  2024       4   
20603                west-bengal  2024       4   

                           District_name  Transaction_count  \
0      north and middle andaman district                442   
1                 south andaman district               5688   
2                      nicobars district                528   
3      north and middle andaman district                825   
4                 south andaman district               9395   
...                  

In [2]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Map_trans_state['State'] = Map_trans_state['State'].str.strip().str.lower()
Map_trans_state['State'] = Map_trans_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Map_Transaction_State(
        State varchar(100),
        Year int,
        Quater int,
        District_name varchar(100),
        Transaction_count float,
        transaction_amount float)
        """
cursor.execute(query1)

In [8]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Map_trans_state.index:
    row = Map_trans_state.loc[index]
    row = row.values
    row = row[0],float(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Map_Transaction_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [3]:
#doing data cleaning for Map_trans_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Map_trans_state.isna().sum()
Map_trans_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['District_name']
dummy_col = ['Year','Quater','State']
encoder = LabelEncoder()
Map_trans_state['encoded_district_name'] = encoder.fit_transform(Map_trans_state['District_name'])
Map_trans_state = pd.get_dummies(Map_trans_state,columns=dummy_col,dtype=int)
#third removing outliners
Map_trans_state['transaction_count_std']=(Map_trans_state['Transaction_count']-Map_trans_state['Transaction_count'].mean())/Map_trans_state['Transaction_count'].std()
Map_trans_state['transaction_amount_std']=(Map_trans_state['Transaction_amount']-Map_trans_state['Transaction_amount'].mean())/Map_trans_state['Transaction_amount'].std()
condition1 = Map_trans_state['transaction_count_std']<=3
condition2 = Map_trans_state['transaction_count_std']>=-3
condition3 = Map_trans_state['transaction_amount_std']<=3
condition4 = Map_trans_state['transaction_amount_std']>=-3
Map_trans_state = Map_trans_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Map_trans_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20604 entries, 0 to 20603
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               20604 non-null  object 
 1   Year                20604 non-null  object 
 2   Quater              20604 non-null  int64  
 3   District_name       20604 non-null  object 
 4   Transaction_count   20604 non-null  int64  
 5   Transaction_amount  20604 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 965.9+ KB
                           District_name  Transaction_count  \
0      north and middle andaman district                442   
1                 south andaman district               5688   
2                      nicobars district                528   
3      north and middle andaman district                825   
4                 south andaman district               9395   
...                                  ...                ..

In [11]:
#plotting aggregated_transaction_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Map_trans_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Transaction_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
#This is for map transaction india 2018 data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/transaction/hover/country/india/2018"
Map_trans_2018_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'State_name':[], 'Transaction_count':[], 'Transaction_amount':[]}

for i in Map_trans_2018_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['hoverDataList']:
        Name=z['name']
        count=z['metric'][0]['count']
        amount=z['metric'][0]['amount']
        clm['State_name'].append(Name)
        clm['Transaction_count'].append(count)
        clm['Transaction_amount'].append(amount)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Map_trans_2018=pd.DataFrame(clm)
print(Map_trans_2018)

     Quater         State_name  Transaction_count  Transaction_amount
0         1         puducherry             104212        1.658260e+08
1         1         tamil nadu            6726622        1.126156e+10
2         1      uttar pradesh           12537805        1.393997e+10
3         1     madhya pradesh            8025395        8.681603e+09
4         1     andhra pradesh            9039585        1.199628e+10
..      ...                ...                ...                 ...
139       4    jammu & kashmir            2721310        5.664720e+09
140       4                goa             514755        1.153005e+09
141       4  arunachal pradesh             176429        5.040400e+08
142       4              delhi           29187719        3.972286e+10
143       4        uttarakhand            3178761        5.654272e+09

[144 rows x 4 columns]
